In [3]:
from response_curves import generate_response_curve, parameter_bounds, sample_curves
from auc_tools import standard_auc, monte_carlo_auc
import numpy as np
import pandas as pd

In [4]:
# Generate actual response curves using the sampled parameters
response_curves_set = []
sampled_curves = sample_curves(parameter_bounds)

x_arr = np.linspace(0, 200, 1000)  # Define the x values for the response curves

for num_participants in range(5, 42, 5):  # Participants from 5 to 41 with step size of 5
    for num_datapoints in range(4, 11):  # Datapoints from 4 to 10
        for curve_type, param_list in sampled_curves.items():
            for index, params in enumerate(param_list):
                # Generate the true response curve
                response, support_start, support_end = generate_response_curve(
                    x_arr, curve_type=curve_type, params=params, threshold=0.01
                )
                
                # Define gridpoints within the active support
                gridpoints = np.linspace(support_start, support_end, num_datapoints)
                
                # Evaluate the response at the gridpoints
                true_values = np.interp(gridpoints, x_arr, response)
                
                # Generate random values around the true values with 20% standard deviation
                random_values = np.random.normal(
                    loc=true_values, scale=true_values * 0.2, size=(num_participants, len(gridpoints))
                )
                
                # Calculate mean and standard deviation of the responses
                mean_responses = random_values.mean(axis=0)
                std_responses = random_values.std(axis=0)
                
                # Append the result to the response_curves_set
                response_curves_set.append({
                    'index': index,
                    'curve_type': curve_type,
                    'parameters': params,
                    'x_vals': gridpoints,
                    'num_participants': num_participants,
                    'mean_responses': mean_responses,
                    'std_responses': std_responses
                })




In [33]:
#This part we should be able to run in a loop, sweeping over number of participants and datapoints

results = {
    "grid_points_in_graph": [],
    "participants": [],
    "curve_type": [],
    "parameters": [],
    "standard_auc_mean": [],
    "standard_auc_lower_bound": [],
    "standard_auc_upper_bound": [],
    "standard_auc_std_bailer": [],
    "mc_auc_mean": [],
    "mc_auc_std": []

}

for ctr, curve in enumerate(response_curves_set):
    current_grid, means, stds = curve['x_vals'], curve['mean_responses'], curve['std_responses']

    #Now we want to calculate the AUC with the established method and the Monte Carlo method
    standard_auc_mean, standard_auc_bailer_std, auc_lower_bound, auc_upper_bound = standard_auc(current_grid, means, stds, curve['num_participants'])

    # Calculate the new AUC using the Monte Carlo method
    mc_auc_mean, mc_auc_std = monte_carlo_auc(current_grid, means, stds, n_sim=1000)

    # Create a pandas DataFrame to store the results
    results["standard_auc_mean"].append(standard_auc_mean)
    results["grid_points_in_graph"].append(len(current_grid))
    results["participants"].append(curve['num_participants'])
    results["curve_type"].append(curve['curve_type'])
    results["parameters"].append(curve['parameters'])       
    results["standard_auc_std_bailer"].append(standard_auc_bailer_std)
    results["standard_auc_lower_bound"].append(auc_lower_bound)
    results["standard_auc_upper_bound"].append(auc_upper_bound)
    results["mc_auc_mean"].append(mc_auc_mean)
    results["mc_auc_std"].append(mc_auc_std)
    if ctr % 100 == 0:
        print(f"Processed {ctr} curves out of {len(response_curves_set)}")

# Assume df is your DataFrame
true_auc = 1.0

df = pd.DataFrame(results) 

# Add relative error column
df["mc_relative_error"] = abs(df["mc_auc_mean"] - true_auc) / true_auc

# Add binary coverage column
df["mc_coverage"] = (
    (df["mc_auc_mean"] - 1.96 * df["mc_auc_std"] <= true_auc) &
    (df["mc_auc_mean"] + 1.96 * df["mc_auc_std"] >= true_auc)
).astype(int)

df['standard_auc_simple_std'] = (df['standard_auc_mean']-df['standard_auc_lower_bound']+df['standard_auc_upper_bound']-df['standard_auc_mean'])/2

# Add binary coverage column
df["standard_coverage"] = (
    (df["standard_auc_mean"] - 1.96 * df["standard_auc_simple_std"] <= true_auc) &
    (df["mc_auc_mean"] + 1.96 * df["standard_auc_simple_std"] >= true_auc)
).astype(int)




Processed 0 curves out of 3920
Processed 100 curves out of 3920
Processed 200 curves out of 3920
Processed 300 curves out of 3920
Processed 400 curves out of 3920
Processed 500 curves out of 3920
Processed 600 curves out of 3920
Processed 700 curves out of 3920
Processed 800 curves out of 3920
Processed 900 curves out of 3920
Processed 1000 curves out of 3920
Processed 1100 curves out of 3920
Processed 1200 curves out of 3920
Processed 1300 curves out of 3920
Processed 1400 curves out of 3920
Processed 1500 curves out of 3920
Processed 1600 curves out of 3920
Processed 1700 curves out of 3920
Processed 1800 curves out of 3920
Processed 1900 curves out of 3920
Processed 2000 curves out of 3920
Processed 2100 curves out of 3920
Processed 2200 curves out of 3920
Processed 2300 curves out of 3920
Processed 2400 curves out of 3920
Processed 2500 curves out of 3920
Processed 2600 curves out of 3920
Processed 2700 curves out of 3920
Processed 2800 curves out of 3920
Processed 2900 curves out 

In [ ]:
df.to_csv('auc_results.csv', index=False)